DataAugumentationを行う際は、testデータとtrainデータに分けてから行う。<br>
DataAugumentaionを先に行ってしまうと、似た画像がtestデータに存在することになり**leakage**が生じる。

### fine-tuning
学習済みのネットワークを利用して、追加で学習を行う。
- なぜうまくいくのか。
どんな分類タスクでも、畳み込み層(浅い層)で検出する特徴は共通。
- 学習の工夫
学習率を通常の100分の1ほどにしておく
- 学習の仕方
**全結合層のみ**乱数で初期化してそれ以外は学習済みを使う。重みは固定する。<br>
判別するクラス数がタスクによって違うためこれは必須。<br>
一部の重みを固定して必要なデータ数を削減することもできる。


In [2]:
# 必要なライブラリの読み込み
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizers
from chainer.datasets import tuple_dataset, TransformDataset
from chainer.training import extensions
from chainer import optimizers, serializers, training, iterators

/home/inouelab/.conda/envs/zukapy35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# pickle形式のファイルを読み込む関数
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        list = pickle.load(fo, encoding='bytes')
    return list

In [4]:
# ダウンロードしたファイルはpickle形式で保存されているため、扱いやすいように読み込む
train = unpickle('train.pickle')
test = unpickle('test.pickle')
label = unpickle('label.pickle')

In [5]:
# 訓練用データ(2500件)
# テスト用データ(500件)
N_train = len(train)
N_test = len(test)
X_train = train['data']
X_test = test['data']
y_train = train['label']
y_test = test['label']

In [10]:
class PretrainedVGG16(chainer.Chain):
    def __init__(self,n_class, lossfunction = F.softmax_cross_entropy, accfunction = F.accuracy):
        super(PretrainedVGG16,self).__init__()
        with self.init_scope():
            self.base = L.VGG16Layers() #基本はVGG16のネットワークを利用する。このfc7の出力を新たに定義したfc8の入力とする。
            self.new_fc8 = L.Linear(None,n_class)
            self.lossfun = lossfunction # 誤差関数を定義
            self.accfun = accfunction
            
    def __call__(self,x,t): #入力データと教師データ
        # 学習をしないとき､勾配の情報を保持しないようにしてメモリを節約
        with chainer.using_config('enable_backprop', False):
                x = np.asarray(x, dtype=np.float32)
        h = F.relu(self.new_fc8(self.base(x, layers=['fc7'])['fc7']))
        y = self.new_fc8(h)
        return self.lossfun(y,t) # 誤差関数を返す。CNNの章ではここを全結合層の出力にして性能評価を外で行なっていた。

In [ ]:
model = PretrainedVGG16(n_class=5)

In [ ]:
model.base.disable_update()

In [ ]:
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)
batchsize = 100
n_epoch = 1 # 簡単に動作確認をするため､epochを1に設定